See function get_tables at bottom for a single function that returns a pandas dataframe after having scraped the requested data from is-academia. 

All the stuff before that function is explantory text documenting the process of understading the request format to use to the the data we wanted.

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

base_url = 'http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247'

When making a request through the form, the following request is made:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_c_langue=&ww_i_reportModelXsl=133685270&zz_x_UNITE_ACAD=Informatique&ww_x_UNITE_ACAD=249847&zz_x_PERIODE_ACAD=&ww_x_PERIODE_ACAD=null&zz_x_PERIODE_PEDAGO=&ww_x_PERIODE_PEDAGO=null&zz_x_HIVERETE=&ww_x_HIVERETE=null&dummy=ok

The params are:  
ww_b_list:1   
ww_i_reportmodel:133685247  
ww_c_langue:  
ww_i_reportModelXsl:133685270  
zz_x_UNITE_ACAD:Informatique  
ww_x_UNITE_ACAD:249847  
zz_x_PERIODE_ACAD:  
ww_x_PERIODE_ACAD:null  
zz_x_PERIODE_PEDAGO:  
ww_x_PERIODE_PEDAGO:null  
zz_x_HIVERETE:  
ww_x_HIVERETE:null  
dummy:ok  

Notice we have specified zz_x_UNITE_ACAD:Informatique i.e. computer science

This gives us a list of all the different student lists for computer science.

This request allows us to select a list and display the students:

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=71454914&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=null&ww_x_HIVERETE=null

The params are:  
ww_x_GPS:71454914  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:null  
ww_x_HIVERETE:null  


For the first question we need to filter on Bachelor, the date and the semestre
Here is the request for bachelor semestre 1 2004-2005:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225262&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=249108&ww_x_HIVERETE=null

ww_x_GPS:2225262  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:249108  
ww_x_HIVERETE:null  

And for bachelor semestre 6 2004-2005:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225150&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=942175&ww_x_HIVERETE=null
ww_x_GPS:2225150  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Remark: we seem to have standard values for ww_x_PERIODE_PEDAGO: 

In [20]:
bachelor_semestre = {1:249108, 6:942175}


Now lets look at how changing the academic period affects the params:
Bachelor semestre 6 2005-2006:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225237&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=942175&ww_x_HIVERETE=null

ww_x_GPS:2225237  
ww_i_reportModel:133685247  # Seems to be invariant  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  # Section: Computer Science  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:942175  # Semestre  
ww_x_HIVERETE:null  

As expected ww_x_PERIODE_PEDAGO is 942175  
The only value that has changed is ww_x_GPS.

Bachelor Seestre 6 2006-2007:  
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2225324&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=942175&ww_x_HIVERETE=null   
ww_x_GPS:2225324  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Again only ww_x_GPS has changed. No obvious pattern, expected that it seems to be an upwards trend as we increase the year. Let's check this. 
As this would be sufficient de filter FROM a specific date.

Bachelor 6 2014-2015:  
ww_x_GPS:1378362238  # OK

BAchelor 6 2016-2017:  
ww_x_GPS:1744378039 # OK the trend seems to hold



Let's look at how to differentiate between bachelor and master
Master 1 2015-2016:
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=1897033225&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=null&ww_x_PERIODE_PEDAGO=2230106&ww_x_HIVERETE=null  
ww_x_GPS:1897033225  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:2230106  
ww_x_HIVERETE:null  

The value that has changed is ww_x_PERIODE_PEDAGO to 2230106  
We can guess that all master 1 students will have this value, whatever the year. Let's check

Master 1 2013 2014  
ww_x_GPS:1378438423  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:null  
ww_x_PERIODE_PEDAGO:2230106  
ww_x_HIVERETE:null  

Great. The theory holds.



Let's try filtering on the year from their form.
Master 1 2016-2017
http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=2021044028&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_ACAD=355925344&ww_x_PERIODE_PEDAGO=2230106&ww_x_HIVERETE=null
ww_x_GPS:2021044028  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:355925344  
ww_x_PERIODE_PEDAGO:2230106  
ww_x_HIVERETE:null  

Interesting, we have a value for ww_x_PERIODE_ACAD. Let's check this represents the year

Bachelor 6 2016-2017:  
ww_x_GPS:1744378039  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:355925344  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Ok great this represents the year. What logic is behind this param ?
Let's go down a year:

BAchelor 6 2015-2016: 
ww_x_GPS:1650772010  
ww_i_reportModel:133685247  
ww_i_reportModelXsl:133685270  
ww_x_UNITE_ACAD:249847  
ww_x_PERIODE_ACAD:213638028  
ww_x_PERIODE_PEDAGO:942175  
ww_x_HIVERETE:null  

Bachelor 6 2014-2015  
ww_x_PERIODE_ACAD:213637922

Ok it looks pretty random, we're just going to get the values for the years we need

In [21]:
# Values for ww_x_PERIODE_ACAD
years = {'2016-2017':355925344,
         '2015-2016':213638028,
         '2014-2015':213637922,
         '2013-2014':213637754,
         '2012-2013':123456101,
         '2011-2012':123455150,
         '2010-2011':39486325,
         '2009-2010':978195,
         '2008-2009':978187,
         '2007-2008':978181 ,     
        }

It also seems there is no obvious where to get ww_x_GPS. Note however that if we request all the lists this param is set to -1. This will do nicely.

In [22]:
def get_data(semestre, year):
    params = {
                'ww_x_GPS':str(-1),  # This gives us all results
                'ww_i_reportModel':133685247,  # Always the same
                'ww_i_reportModelXsl':133685270,  # Always the same
                'ww_x_UNITE_ACAD':249847,  # The section, we also always looking at computer sicence
                'ww_x_PERIODE_ACAD':str(year),  # The school year, ex: 2016-2017
                'ww_x_PERIODE_PEDAGO':str(semestre),  # The semestre, ex Bachelor 1
                'ww_x_HIVERETE':'null'  # Unknown param, always seems to be null
             }
    url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS={ww_x_GPS}&ww_i_reportModel={ww_i_reportModel}&ww_i_reportModelXsl={ww_i_reportModelXsl}&ww_x_UNITE_ACAD={ww_x_UNITE_ACAD}&ww_x_PERIODE_ACAD={ww_x_PERIODE_ACAD}&ww_x_PERIODE_PEDAGO={ww_x_PERIODE_PEDAGO}&ww_x_HIVERETE={ww_x_HIVERETE}'.format(**params)
    r = requests.get(url)
    return r

In [23]:
# Lets test our function
# assert get_data(bachelor_semestre[1], years['2016-2017']).status_code == 200

In [24]:
# f = open('samplehtml.html','w')
# f.write(get_data(bachelor_semestre[1], years['2016-2017']).text)

Let's start having a look at how we can parse this data:
The data we want is basically in an html table.


In [25]:
data = get_data(bachelor_semestre[1], years['2016-2017'])
soup = BeautifulSoup(data.text, 'html.parser')

In [26]:
# soup

In [27]:
len(soup.table)

237

In [28]:
table = soup.table

In [29]:
data_list = []
rows = table.find_all('tr')
for i,row in enumerate(rows):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    cols = cols[:-1]  # There' s an empty column at the end
    data_list.append(cols)
df = pd.DataFrame(data_list[2:],columns=['Civilité','Nom Prénom','Orientation Bachelor','Orientation Master','Spécialisation','Filière opt.','Mineur','Statut','Type Echange','Ecole Echange','No Sciper'])


In [30]:
df.head()



,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Abbey Alexandre,,,,,,Présent,,,235688
1,Monsieur,Ahn Seongho,,,,,,Présent,,,274015
2,Madame,Alemanno Sara,,,,,,Présent,,,268410
3,Monsieur,Althaus Luca,,,,,,Présent,,,271464
4,Monsieur,Assi Karim,,,,,,Présent,,,274518


Ok so let's put all of this together in a single function. This function will get the data and return it in a pandas dataframe

In [31]:
def get_table(semestre, year, student_type):
    """
    @param semestre: Int represented the semestre number: for example put 1 for semestre 1
    @param year: String representing the academic year: for example '2016-2017'
    @param student type: String either 'bachelor' or 'master'
    @return: pandas dataframe with corresponding data from Is-Academia
    """
    
    years = {'2016-2017':355925344,
         '2015-2016':213638028,
         '2014-2015':213637922,
         '2013-2014':213637754,
         '2012-2013':123456101,
         '2011-2012':123455150,
         '2010-2011':39486325,
         '2009-2010':978195,
         '2008-2009':978187,
         '2007-2008':978181 ,     
        }
    bachelor_semestre = {'1':249108, '2':249114, '3':942155, '4':942163, '5':942120, '5b':2226768, '6':942175, '6b':2226785}
    master_semestre = {'1':2230106, '2':942192, '3':2230128, 'Autumn Project':249127, 'Spring Project':3781783}
    
    if year in years.keys():
        year_code = years[year]
    else:
        raise Exception('Invalid year selected')
        
    if student_type == 'bachelor':
        
        if semestre in ['1', '2', '3', '4', '5', '5b', '6', '6b']:
            semestre_code = bachelor_semestre[semestre]
        else:
            raise Exception('Invalid semestre selected')
            
    elif student_type == 'master':
            
        if semestre in ['1', '2', '3', 'Autumn Project', 'Spring Project']:
            semestre_code = master_semestre[semestre]
        else:
            raise Exception('Invalid semestre selected')
        
    else:
        raise Exception('Invalid student type. Please choose either bachelor or master')
    
    params = {
                'ww_x_GPS':str(-1),  # This gives us all results
                'ww_i_reportModel':133685247,  # Always the same
                'ww_i_reportModelXsl':133685270,  # Always the same
                'ww_x_UNITE_ACAD':249847,  # The section, we also always looking at computer sicence
                'ww_x_PERIODE_ACAD':str(year_code),  # The school year, ex: 2016-2017
                'ww_x_PERIODE_PEDAGO':str(semestre_code),  # The semestre, ex Bachelor 1
                'ww_x_HIVERETE':'null'  # Unknown param, always seems to be null
             }
    url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS={ww_x_GPS}&ww_i_reportModel={ww_i_reportModel}&ww_i_reportModelXsl={ww_i_reportModelXsl}&ww_x_UNITE_ACAD={ww_x_UNITE_ACAD}&ww_x_PERIODE_ACAD={ww_x_PERIODE_ACAD}&ww_x_PERIODE_PEDAGO={ww_x_PERIODE_PEDAGO}&ww_x_HIVERETE={ww_x_HIVERETE}'.format(**params)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.table
    data_list = []
    rows = table.find_all('tr')
    for i,row in enumerate(rows):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        cols = cols[:-1]  # There' s an empty column at the end
        data_list.append(cols)
    columns=['Civilité','Nom Prénom','Orientation Bachelor','Orientation Master','Spécialisation','Filière opt.','Mineur','Statut','Type Echange','Ecole Echange','No Sciper']
    df = pd.DataFrame(data_list[2:],columns=columns)
    return df


In [32]:
# test
get_table('1','2016-2017', 'master')

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Adams Marc Alain,,,,,,Présent,,,271331
1,Monsieur,Adamsons Valdis,,,,,,Présent,,,273101
2,Monsieur,Agapiou Stylianos,,,,,,Présent,,,267543
3,Monsieur,Aiulfi Loris Sandro,,,,,,Présent,,,202293
4,Monsieur,Alfonso Peterssen Alfonso,,,Foundations of Software,,,Présent,,,228219
5,Monsieur,Angerand Grégoire Georges Jacques,,,,,,Présent,,,212464
6,Monsieur,Anongba Varela Dario Abougnan,,,Software Systems,,,Présent,,,226371
7,Monsieur,Aquil Mohammad Amimul Ihsan,,,Foundations of Software,,,Présent,,,273107
8,Monsieur,Bao Junze,,,,,,Présent,,,266983
9,Monsieur,Barbera Antonino,,,,,,Présent,,,273118


To avoid doing multiple IS-Academia requests, let's just save all the records we need in a local folder.

In [41]:
directory = os.path.realpath ('__file__') + '\\IS-Academia_Data\\'
years = ['2016-2017', '2015-2016', '2014-2015', '2013-2014', '2012-2013', '2011-2012', '2010-2011', '2009-2010', '2008-2009', '2007-2008']
bachelor_semesters = ['1', '2', '3', '4', '5', '5b', '6', '6b']
master_semesters = ['1', '2', '3', 'Autumn Project', 'Spring Project']

for year in years:
    for semester in bachelor_semesters:
        writer_bachelor = pd.ExcelWriter('bachelor_' + year + '_' + semester + '.xlsx', engine='xlsxwriter')
        get_table (semester, year, 'bachelor').to_excel (writer_bachelor, sheet_name = 'Main Sheet')
        writer_bachelor.save ()
        writer_bachelor.close ()
    for semester in master_semesters:
        writer_master = pd.ExcelWriter('master_' + year + '_' + semester + '.xlsx', engine='xlsxwriter')
        get_table (semester, year, 'master').to_excel (writer_master, sheet_name = 'Main Sheet')
        writer_master.save ()
        writer_master.close ()

## Exercice 1 : How much time does it take to complete Bachelor degree ?

As we want all the students that have gone through all the semesters (6), and as we want this since 2007, we need a small loop to merge data frames about either semester 1 to 6.

Basically, all we have to do is first load all the semesters students Sciper number from 2007-2008 to 20xx-20xx (we can assume that a bachelor degree cannot be completed in under 3 years, hence, we don't need the 2015 records for semester 1 for example), create a column containing the number 1 (attempts) to sum later the count of semesters, then concatenate all of it for each semester, sum the semester attempts, and inner merge all semesters to get rid of those who didn't attend semester 6. The last thing we'll have to do is to sum the 'Attempts' columns, and groupby "civilité" to compute the mean for each gender.

What we are basically doing, is that we'll count the number of semester attended by each student, and do a mean.

To avoid requesting data everytime on IS-Academia for various operations and debugging, let's create a record for every semester, degree type, and year.

In [77]:
data_storage = {}

So first, let us build a function that computes all students that attended a given semester in a given degree type (bachelor or master)

In [79]:
def computeSemesterStudentsBetween (semester, degree_type, fromYear2k, toYear2k):

    #We'll build a frame list to concatenate afterwards
    frame_list = []
    
    #The year interval we want
    interval = toYear2k - fromYear2k + 1
    
    #The name of our attempt, which is basically 'Attempts ' plus the semester number
    attempts = 'Attempts ' + str(semester)

    for i in range (interval):

        #Take the lower bound year, in order to grab the right information on IS-Academia
        #Basically we are just building a serie of '20xx-20yy' for the scrapping, where xx - yy = 1
        lowerBoundYear = i + fromYear2k
        upperBoundYear = lowerBoundYear + 1

        stringLowerBoundYear = '20'
        stringUpperBoundYear = '20'

        if lowerBoundYear < 10:
            stringLowerBoundYear += '0'

        if upperBoundYear < 10:
            stringUpperBoundYear += '0'

        stringLowerBoundYear += str (lowerBoundYear)
        stringUpperBoundYear += str (upperBoundYear)

        #So now 'year' is something like '2016-2017'
        year = stringLowerBoundYear + '-' + stringUpperBoundYear
        
        #Compute the key for data record
        key = str (degree_type) + ' ' + str (semester) + ' ' + str (lowerBoundYear)

        #Grab this year's information using the function we built earlier
        if key not in data_storage:
            data_storage ['key'] = get_table (semester, year, degree_type)
            
        t = data_storage ['key']
        
        #Adding the attempt column, to be summed after the groupby
        t[attempts] = 1

        #Adding the current data frame to the list to concatenate
        frame_list.append (t[['No Sciper', 'Civilité', attempts]])
        
    #Finally, we just concatenate all the gathered data frames into one big, and we sum the attempts.
    output = pd.concat (frame_list)
    output = output.groupby ('No Sciper').agg ({'No Sciper': 'first', 'Civilité': 'first', attempts: sum})[['Civilité', attempts]]
    
    return output

Now, let us try out this splendid function.

In [81]:
semester_1 = computeSemesterStudentsBetween (1, 'bachelor', 7, 13)
semester_1

,Civilité,Attempts 1
No Sciper,,
147008,Monsieur,1
161212,Monsieur,1
169569,Monsieur,1
169731,Monsieur,1
169795,Monsieur,1
170509,Monsieur,1
171195,Monsieur,1
171568,Monsieur,1
171619,Monsieur,1


Alright ! So now we seem to have a good input for semester 1, with 824 students for the Informatics bachelor degree. Let's now compute the other semesters.

In [82]:
all_semesters = semester_1

for i in range (5):
    all_semesters = pd.merge (all_semesters, pd.DataFrame (computeSemesterStudentsBetween (i + 2, 'bachelor', 7, 14 + (i + 1) % 2) ['Attempts ' + str(i + 2)]), left_index=True, right_index=True, how='inner')
    
all_semesters.head ()

,Civilité,Attempts 1,Attempts 2,Attempts 3,Attempts 4,Attempts 5,Attempts 6
No Sciper,,,,,,,
147008,Monsieur,1,1,1,1,1,1
169569,Monsieur,1,1,1,1,1,1
169731,Monsieur,1,1,2,2,1,1
169795,Monsieur,1,1,1,1,3,2
171195,Monsieur,1,1,1,1,1,1


This looks promising. Let's now have a look on the global mean by student.

In [83]:
total_attempts = all_semesters [['Civilité']]
total_attempts ['Total Attempts'] = 0

for i in range(6):
    total_attempts ['Total Attempts'] += all_semesters ['Attempts ' + str(i + 1)]
    
total_attempts.head ()

C:\Users\Germanicus\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Germanicus\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Civilité,Total Attempts
No Sciper,,
147008,Monsieur,6
169569,Monsieur,6
169731,Monsieur,8
169795,Monsieur,9
171195,Monsieur,6


Just a small check, to see if no student graduated in less than 6 months

In [84]:
total_attempts ['Total Attempts'].min ()

6

Now let's compute the mean months duration for all students

In [85]:
total_attempts ['Total Attempts'].mean () * 5

42.30283911671924

And finally, compute the mean by gender :

In [86]:
total_attempts.groupby ('Civilité') ['Total Attempts'].mean () * 5

Civilité
Madame      40.000000
Monsieur    42.466216
Name: Total Attempts, dtype: float64

We can conclude that women are more likely to graduate faster than men.